# Sklearn, XGBoost

## sklearn.ensemble.RandomForestClassifier

In [2]:
from sklearn import ensemble , cross_validation, learning_curve, metrics 

import numpy as np
import pandas as pd
import xgboost as xgb

ImportError: No module named 'xgboost'

In [10]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


### Данные

Задача на kaggle: https://www.kaggle.com/c/bioresponse

Данные: https://www.kaggle.com/c/bioresponse/data

По данным характеристикам молекулы требуется определить, будет ли дан биологический ответ (biological response).

Признаки нормализаваны.

Для демонстрации используется обучающая выборка из исходных данных train.csv, файл с данными прилагается.

In [11]:
bioresponce = pd.read_csv('bioresponse.csv', header=0, sep=',')

In [12]:
bioresponce.head()

,Activity,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,D1767,D1768,D1769,D1770,D1771,D1772,D1773,D1774,D1775,D1776
0,1,0.000000,0.497009,0.10,0.0,0.132956,0.678031,0.273166,0.585445,0.743663,...,0,0,0,0,0,0,0,0,0,0
1,1,0.366667,0.606291,0.05,0.0,0.111209,0.803455,0.106105,0.411754,0.836582,...,1,1,1,1,0,1,0,0,1,0
2,1,0.033300,0.480124,0.00,0.0,0.209791,0.610350,0.356453,0.517720,0.679051,...,0,0,0,0,0,0,0,0,0,0
3,1,0.000000,0.538825,0.00,0.5,0.196344,0.724230,0.235606,0.288764,0.805110,...,0,0,0,0,0,0,0,0,0,0
4,0,0.100000,0.517794,0.00,0.0,0.494734,0.781422,0.154361,0.303809,0.812646,...,0,0,0,0,0,0,0,0,0,0


В этой задаче требуется предсказать, будет ли дан биологический ответ на основе данных о молекуле. Мы уже работали с этими данными ранее, когда изучали random forest.  Помним, что в первом столбце у нас дана метка класса: 1 означает, что биологический ответ будет дан, 0 означает обратное. И также нам доступны столбцы с параметрами молекулы. 

 Давайте теперь отделим данные от метки класса. 

In [13]:
bioresponce_target = bioresponce.Activity.values

In [14]:
bioresponce_data = bioresponce.iloc[:, 1:]

### Модель RandomForestClassifier

#### Зависимость качества от количесвта деревьев

И можем переходить к построению моделей. И случайный лес, и градиентный бустинг являются композициями алгоритмов. Понятно, что их качество зависит от количества простых моделей, над которыми они строятся. Давайте проанализируем, как изменяется качество алгоритма в зависимости от количества деревьев, над которыми они построены. Начнем со случайного леса. Сначала зададим количество деревьев. Давайте смотреть на одно дерево и далее на количество деревьев от 10 до 55 с шагом 5. Что мы хотим понять? Мы хотим понять, как будет меняться качество в зависимости от того, сколько деревьев мы разрешили строить алгоритму. Для этого давайте пройдемся в цикле по заданному количеству деревьев и будем оценивать качество с помощью функции cross_val_score. При этом мы будем делать кросс-валидацию на три фолда и будем использовать метрику accuracy. Вот давайте получим результаты этого скоринга. 

In [15]:
n_trees = [1] + list(range(10, 55, 5)) 

In [16]:
%%time
scoring = []
for n_tree in n_trees:
    estimator = ensemble.RandomForestClassifier(n_estimators = n_tree, min_samples_split=5, random_state=1)
    score = cross_validation.cross_val_score(estimator, bioresponce_data, bioresponce_target, 
                                             scoring = 'accuracy', cv = 3)    
    scoring.append(score)
scoring = np.asmatrix(scoring)

CPU times: user 18.4 s, sys: 1.27 s, total: 19.7 s
Wall time: 22.1 s


In [ ]:
scoring

In [ ]:
pylab.plot(n_trees, scoring.mean(axis = 1), marker='.', label='RandomForest')
pylab.grid(True)
pylab.xlabel('n_trees')
pylab.ylabel('score')
pylab.title('Accuracy score')
pylab.legend(loc='lower right')

#### Кривые обучения для деревьев большей глубины

In [ ]:
%%time
xgb_scoring = []
for n_tree in n_trees:
    estimator = xgb.XGBClassifier(learning_rate=0.1, max_depth=5, n_estimators=n_tree, min_child_weight=3)
    score = cross_validation.cross_val_score(estimator, bioresponce_data, bioresponce_target, 
                                             scoring = 'accuracy', cv = 3)    
    xgb_scoring.append(score)
xgb_scoring = np.asmatrix(xgb_scoring)

In [ ]:
xgb_scoring

In [2]:
pylab.plot(n_trees, scoring.mean(axis = 1), marker='.', label='RandomForest')
pylab.plot(n_trees, xgb_scoring.mean(axis = 1), marker='.', label='XGBoost')
pylab.grid(True)
pylab.xlabel('n_trees')
pylab.ylabel('score')
pylab.title('Accuracy score')
pylab.legend(loc='lower right')

NameError: name 'pylab' is not defined

#### **Если Вас заинтересовал xgboost:**
python api: http://xgboost.readthedocs.org/en/latest/python/python_api.html

установка: http://xgboost.readthedocs.io/en/latest/build.html